In [ ]:
!pip uninstall apache-beam -y && pip install -U pandas polars pyarrow narwhals 

In [ ]:
import pandas as pd
import polars as pl

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True

In [ ]:
from typing import Any
import narwhals as nw

def q11(
    partsupp_ds_raw: Any,
    nation_ds_raw: Any,
    supplier_ds_raw: Any,
) -> Any:

    nation_ds = nw.from_native(nation_ds_raw)
    partsupp_ds = nw.from_native(partsupp_ds_raw)
    supplier_ds = nw.from_native(supplier_ds_raw)

    
    var1 = "GERMANY"
    var2 = 0.0001

    q1 = (
        partsupp_ds.join(supplier_ds, left_on="ps_suppkey", right_on="s_suppkey")
        .join(nation_ds, left_on="s_nationkey", right_on="n_nationkey")
        .filter(nw.col("n_name") == var1)
    )
    q2 = q1.select(
        (nw.col("ps_supplycost") * nw.col("ps_availqty")).sum().round(2).alias("tmp")
        * var2
    )

    q_final = (
        q1.with_columns(
            (nw.col("ps_supplycost") * nw.col("ps_availqty"))
            .alias("value")
        )
        .group_by("ps_partkey")
        .agg(
            nw.sum("value")
        )
        .join(q2, how="cross")
        .filter(nw.col("value") > nw.col("tmp"))
        .select("ps_partkey", "value")
        .sort("value", descending=True)
    )

    return nw.to_native(q_final)

In [ ]:
dir_ = "/kaggle/input/tpc-h-data-parquet-s-2/"
nation = dir_ + 'nation.parquet'
supplier = dir_ + 'supplier.parquet'
partsupp = dir_ + 'partsupp.parquet'

In [ ]:
IO_FUNCS = {
    'pandas': lambda x: pd.read_parquet(x, engine='pyarrow'),
    'pandas[pyarrow]': lambda x: pd.read_parquet(x, engine='pyarrow', dtype_backend='pyarrow'),
    'polars[eager]': lambda x: pl.read_parquet(x),
    'polars[lazy]': lambda x: pl.scan_parquet(x),
}

In [ ]:
results = {}

## pandas via Narwhals

In [ ]:
tool = 'pandas'
fn = IO_FUNCS[tool]
timings = %timeit -o q11(fn(partsupp), fn(nation), fn(supplier))
results[tool] = timings.all_runs

## pandas, pyarrow dtypes, via Narwhals

In [ ]:
tool = 'pandas[pyarrow]'
fn = IO_FUNCS[tool]
timings = %timeit -o q11(fn(partsupp), fn(nation), fn(supplier))
results[tool] = timings.all_runs

## Polars read_parquet

In [ ]:
tool = 'polars[eager]'
fn = IO_FUNCS[tool]
timings = %timeit -o q11(fn(partsupp), fn(nation), fn(supplier))
results[tool] = timings.all_runs

## Polars scan_parquet

In [ ]:
tool = 'polars[lazy]'
fn = IO_FUNCS[tool]
timings = %timeit -o q11(fn(partsupp), fn(nation), fn(supplier)).collect()
results[tool] = timings.all_runs

## Save

In [ ]:
import json
with open('results.json', 'w') as fd:
    json.dump(results, fd)


In [ ]:
from pprint import pprint

pprint(results)